### Mount drive, unzip stressid.zip, extract audio as .wav


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install moviepy --quiet

import os
from moviepy.editor import VideoFileClip

Mounted at /content/drive


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



In [ ]:
import os, zipfile
from pathlib import Path
import subprocess

ZIP_PATH = "/content/drive/MyDrive/WuHaoAllenCentad/StressID.zip"
UNZIP_DIR = "/content/temp_stressid_unzip"
WAV_AUDIO_DIR = "/content/drive/MyDrive/WuHaoAllenCentad/stress_id_wav_full"

os.makedirs(UNZIP_DIR, exist_ok=True)
os.makedirs(WAV_AUDIO_DIR, exist_ok=True)

# Unzip (fix: mode is 'r' not 'r ')
with zipfile.ZipFile(ZIP_PATH, "r") as z:
    z.extractall(UNZIP_DIR)

print("Unzipped contents:", os.listdir(UNZIP_DIR))

# Point to the actual dataset folder if it exists
DATA_ROOT = Path(UNZIP_DIR) / "StressID Dataset"
if not DATA_ROOT.exists():
    DATA_ROOT = Path(UNZIP_DIR)

# Recursively find videos
video_exts = {".mp4", ".mov", ".mkv", ".avi", ".webm", ".m4v"}
video_files = [p for p in DATA_ROOT.r కాగా("*") if p.suffix.lower() in video_exts]
print(f"Found {len(video_files)} video files under: {DATA_ROOT}")

# Extract audio with ffmpeg (more reliable than moviepy)
count_extracted = count_skipped = errors = 0

for p in video_files:
    rel = p.relative_to(DATA_ROOT)
    out_name = str(rel).replace("/", "__").rsplit(".", 1)[0] + ".wav"
    out_path = Path(WAV_AUDIO_DIR) / out_name

    if out_path.exists() and out_path.stat().st_size > 0:
        count_skipped += 1
        continue

    cmd = ["ffmpeg", "-y", "-i", str(p), "-vn", "-ac", "1", "-ar", "16000", "-acodec", "pcm_s16le", str(out_path)]
    try:
        subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
        count_extracted += 1
    except subprocess.CalledProcessError:
        errors += 1
        out_path.unlink(missing_ok=True)

print(f"Done. Extracted={count_extracted} Skipped={count_skipped} Errors={errors}")
print("WAV folder:", WAV_AUDIO_DIR)


Scanning /content/temp_stressid_unzip for videos to extract as WAV...
Extracting: t6v9_Counting3.mp4 -> t6v9_Counting3.wav
Extracting: t6v9_Counting2.mp4 -> t6v9_Counting2.wav
Extracting: t6v9_Stroop.mp4 -> t6v9_Stroop.wav
Extracting: t6v9_Baseline.mp4 -> t6v9_Baseline.wav
Extracting: t6v9_Math.mp4 -> t6v9_Math.wav
Extracting: t6v9_Video1.mp4 -> t6v9_Video1.wav
Extracting: t6v9_Counting1.mp4 -> t6v9_Counting1.wav
Extracting: t6v9_Relax.mp4 -> t6v9_Relax.wav
Extracting: t6v9_Speaking.mp4 -> t6v9_Speaking.wav
Extracting: t6v9_Video2.mp4 -> t6v9_Video2.wav
Extracting: t6v9_Reading.mp4 -> t6v9_Reading.wav
Extracting: t6v9_Breathing.mp4 -> t6v9_Breathing.wav
Extracting: d4n6_Breathing.mp4 -> d4n6_Breathing.wav
Extracting: d4n6_Counting2.mp4 -> d4n6_Counting2.wav
Extracting: d4n6_Stroop.mp4 -> d4n6_Stroop.wav
Extracting: d4n6_Counting1.mp4 -> d4n6_Counting1.wav
Extracting: d4n6_Relax.mp4 -> d4n6_Relax.wav
Extracting: d4n6_Speaking.mp4 -> d4n6_Speaking.wav
Extracting: d4n6_Baseline.mp4 -> d4n

### Split .wav files into 10s clips with 5s overlap on each end. Truncate trailing audio. (0s-10s, 5s-15s, 10s-20s...)

In [ ]:
INPUT_DIR = EXTRACT_DIR
!pip install moviepy --quiet

from moviepy.editor import VideoFileClip

# create output folder if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# splitting parameters
WINDOW_SIZE = 10  # seconds per clip
STEP = 5          # overlap step (5s → 50% overlap)
VALID_EXTS = {'.mp4', '.avi', '.mov', '.mkv'}

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



In [ ]:
from pydub import AudioSegment
import os

# Define the main directory containing subfolders for each subject
MAIN_INPUT_DIR = "/content/drive/MyDrive/WuHaoAllenCentad/StressID_split_10s"  # Top-level folder containing subjects
OUTPUT_DIR = "/content/drive/MyDrive/WuHaoAllenCentad/StressID__audio_split_10s"  # Where split audio will go

# Create output folder if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# General keywords to skip (files containing these words)
KEYWORDS_TO_SKIP = ["Breathing", "Relax"]

# Audio splitting parameters
WINDOW_SIZE = 10 * 1000  # 10 seconds in milliseconds
STEP = 5 * 1000  # 5 seconds overlap

def split_audio(file_path, output_dir, subject_name):
    base_name = os.path.basename(file_path)
    name, ext = os.path.splitext(base_name)

    # Skip non-mp3 files
    if ext.lower() != '.mp3':
        return

    # Skip files containing any of the keywords to skip
    if any(keyword in base_name for keyword in KEYWORDS_TO_SKIP):
        print(f"Skipping {base_name} as it contains one of the skip keywords.")
        return

    # Prepare subject-specific folder in output directory
    subject_output_dir = os.path.join(output_dir, subject_name)
    os.makedirs(subject_output_dir, exist_ok=True)

    print(f"Processing: {file_path}")

    try:
        # Load the audio file
        audio = AudioSegment.from_mp3(file_path)

        # Split audio into 10s segments with 5s overlap
        start = 0
        clip_idx = 0
        while start + WINDOW_SIZE <= len(audio):
            end = start + WINDOW_SIZE
            subclip = audio[start:end]

            # Define output file path for the subclip
            out_name = f"{name}_s{start//1000}_e{end//1000}.mp3"
            out_path = os.path.join(subject_output_dir, out_name)

            # Save the subclip
            print(f"  -> Saving clip {clip_idx}: {start/1000}s to {end/1000}s as {out_name}")
            subclip.export(out_path, format="mp3")

            clip_idx += 1
            start += STEP  # move by 5s overlap

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Process each subject's folder and its files
for subject_folder in os.listdir(MAIN_INPUT_DIR):
    subject_folder_path = os.path.join(MAIN_INPUT_DIR, subject_folder)

    # Process only if it's a directory (representing a subject)
    if os.path.isdir(subject_folder_path):
        print(f"Processing subject: {subject_folder}")

        # Walk through each file in the subject's folder
        for root, dirs, files in os.walk(subject_folder_path):
            for fname in files:
                file_path = os.path.join(root, fname)
                split_audio(file_path, OUTPUT_DIR, subject_folder)

  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)

  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)

  elif re.match('(flt)p?( \(default\))?$', token):

  elif re.match('(dbl)p?( \(default\))?$', token):



Streaming output truncated to the last 5000 lines.
  -> Saving clip 17: 85.0s to 95.0s as cxj0_Video1_s85_e95.mp3
  -> Saving clip 18: 90.0s to 100.0s as cxj0_Video1_s90_e100.mp3
  -> Saving clip 19: 95.0s to 105.0s as cxj0_Video1_s95_e105.mp3
  -> Saving clip 20: 100.0s to 110.0s as cxj0_Video1_s100_e110.mp3
  -> Saving clip 21: 105.0s to 115.0s as cxj0_Video1_s105_e115.mp3
  -> Saving clip 22: 110.0s to 120.0s as cxj0_Video1_s110_e120.mp3
  -> Saving clip 23: 115.0s to 125.0s as cxj0_Video1_s115_e125.mp3
  -> Saving clip 24: 120.0s to 130.0s as cxj0_Video1_s120_e130.mp3
  -> Saving clip 25: 125.0s to 135.0s as cxj0_Video1_s125_e135.mp3
  -> Saving clip 26: 130.0s to 140.0s as cxj0_Video1_s130_e140.mp3
  -> Saving clip 27: 135.0s to 145.0s as cxj0_Video1_s135_e145.mp3
  -> Saving clip 28: 140.0s to 150.0s as cxj0_Video1_s140_e150.mp3
  -> Saving clip 29: 145.0s to 155.0s as cxj0_Video1_s145_e155.mp3
  -> Saving clip 30: 150.0s to 160.0s as cxj0_Video1_s150_e160.mp3
  -> Saving clip 31

In [ ]:
import os
import shutil

# Define the directory where the clips are saved
INPUT_DIR = "/content/drive/MyDrive/WuHaoAllenCentad/StressID__audio_split_10s"  # Folder containing all the split clips
OUTPUT_DIR = "/content/drive/MyDrive/stress_id_dataset/sorted_split_clips"  # Where to organize clips by user_task

# Create the sorted output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

def organize_by_user_task(input_dir, output_dir):
    # Walk through all the files in the input directory
    for root, dirs, files in os.walk(input_dir):
        for fname in files:
            # Extract user and task info from the filename (assuming format: user_task_sX_eX.mp3)
            base_name = os.path.basename(fname)

            # Assuming the format is something like '4woj_Counting1_s0_e10.mp3'
            # We can split based on underscores and dash to get user and task
            parts = base_name.split('_')

            if len(parts) < 2:
                continue  # Skip if the file doesn't match the expected naming pattern

            user = parts[0]  # User ID (e.g., '4woj')
            task = parts[1]  # Task (e.g., 'Counting1')

            # Create a directory for the user-task if it doesn't exist
            user_task_dir = os.path.join(output_dir, f"{user}_{task}")
            os.makedirs(user_task_dir, exist_ok=True)

            # Define the full path for the file to move
            source_file = os.path.join(root, fname)
            destination_file = os.path.join(user_task_dir, fname)

            # Move the file to the correct directory
            shutil.move(source_file, destination_file)
            print(f"Moved {fname} to {user_task_dir}")

# Run the function to organize the files
organize_by_user_task(INPUT_DIR, OUTPUT_DIR)

Streaming output truncated to the last 5000 lines.
Moved x1q3_Stroop_s25_e35.mp3 to /content/drive/MyDrive/stress_id_dataset/sorted_split_clips/x1q3_Stroop
Moved x1q3_Stroop_s30_e40.mp3 to /content/drive/MyDrive/stress_id_dataset/sorted_split_clips/x1q3_Stroop
Moved x1q3_Stroop_s35_e45.mp3 to /content/drive/MyDrive/stress_id_dataset/sorted_split_clips/x1q3_Stroop
Moved x1q3_Stroop_s40_e50.mp3 to /content/drive/MyDrive/stress_id_dataset/sorted_split_clips/x1q3_Stroop
Moved x1q3_Stroop_s45_e55.mp3 to /content/drive/MyDrive/stress_id_dataset/sorted_split_clips/x1q3_Stroop
Moved x1q3_Stroop_s50_e60.mp3 to /content/drive/MyDrive/stress_id_dataset/sorted_split_clips/x1q3_Stroop
Moved x1q3_Counting3_s0_e10.mp3 to /content/drive/MyDrive/stress_id_dataset/sorted_split_clips/x1q3_Counting3
Moved x1q3_Counting3_s5_e15.mp3 to /content/drive/MyDrive/stress_id_dataset/sorted_split_clips/x1q3_Counting3
Moved x1q3_Counting3_s10_e20.mp3 to /content/drive/MyDrive/stress_id_dataset/sorted_split_clips/x1q